In [2]:
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LogisticRegression
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('/content/diabetes.csv')

train, test = train_test_split(df, test_size=0.2)
print(train.shape)
print(train.isnull().values.any())
print(test.shape)
print(test.isnull().values.any())

(614, 9)
False
(154, 9)
False


In [3]:
train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
226,0,101,76,0,0,35.7,0.198,26,0
504,3,96,78,39,0,37.3,0.238,40,0
684,5,136,82,0,0,0.0,0.640,69,0
66,0,109,88,30,0,32.5,0.855,38,1
427,1,181,64,30,180,34.1,0.328,38,1


In [4]:
test.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
498,7,195,70,33,145,25.1,0.163,55,1
614,11,138,74,26,144,36.1,0.557,50,1
169,3,111,90,12,78,28.4,0.495,29,0
670,6,165,68,26,168,33.6,0.631,49,0
574,1,143,86,30,330,30.1,0.892,23,0


In [5]:
X_train = train.iloc[:,:-1].values
y_train = train.iloc[:,-1:].values
X_test = test.iloc[:,:-1].values
y_test = test.iloc[:,-1:].values

In [6]:
model = LogisticRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
# Use score method to get accuracy of model
score = model.score(X_test, y_test)
print("Accuracy: "+str(score))

Accuracy: 0.7857142857142857


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [8]:
from sklearn.model_selection import cross_validate
results = cross_validate(estimator=model, X=X_train, y=y_train, cv = 5)
results

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example

{'fit_time': array([0.03460979, 0.03911924, 0.04789114, 0.04502726, 0.04738283]),
 'score_time': array([0.00042367, 0.00065851, 0.00065947, 0.00101233, 0.00172281]),
 'test_score': array([0.75609756, 0.79674797, 0.78861789, 0.77235772, 0.76229508])}